# Prompt 测试 Pipeline

prompt 路线：
text -> jsonline -> prompt_template(str) -> prompt -> 函数测试！！！

user prompt是比较复杂且很长的，并有很多冗余指令，选择封装 prompt_template 完成
为了保证结果的准确统一，在添加一个 prompt 之前一定要严格检查，通过以下说明的一些测试

### text2jsonline
提示词交由 deepseek 完成，提示词还得优化测试，下面是测试过的，只不过键名粘贴到 prompt.json 中还需调整

上面是一些我想展示的 prompt template，我想让你帮我编写一行的 json，键名为 prompt，用合适的换行符，使得我打印这个 json 的时候展示的是上面的内容。你可能需要以下的转换
转义 JSON 大括号：
  - 将 JSON 数据中的 { 改为 {{，} 改为 }}。实现转义，防止后面使用 format 方法将 json 数据的{}识别为占位符
保留动态变量：
  - 需要替换的变量（如 {domain}、{description}）保持单括号不变。

例如下面就是我送给 llm ，他返回的一个 jsonline

In [4]:
jsonline = {
    "prompt": "The Bayesian Optimization is in progress and is currently at iteration {iteration}.\nYou generated the following comments in the previous iterations and your suggestion points were subsequently evaluated and appended to the dataset:\n{comment_history}\n\nNew experimental data has been obtained giving us currently a dataset of {iteration} experiments:\n{trial_data}\n\n**Task**:\n - Reflect on your previous hypotheses and on the entire data.\n - What do we know from the data?\n - Continue live commenting on the optimization progress.\n - What are the subspaces that maximize {target} the most?\n - What are the subspaces that are detrimental and minimize {target}?\n - Improve your previous hypotheses for maximizing {target} in the light of the new comment and learnings. Feel free to also discard some hypotheses that have proven to be performing consistently poorly in previous iterations, and/or propose new ones. Additionally for each hypothesis, give a single point {constraint} you want the Bayesian Optimizer to try.\n - Important: Only provide your response in the exact JSON response format as same as comment in the first round, without any additional syntax or libraries.\n\n```json\n{{\n  \"comment\": \"...\",\n  \"hypotheses\": [\n    {{\n      \"name\": \"...\",\n      \"rationale\": \"...\",\n      \"confidence\": \"...\",\n      \"points\": [{{...}}]\n    }}\n  ]\n}}\n```"
}
print(jsonline['prompt'])

The Bayesian Optimization is in progress and is currently at iteration {iteration}.
You generated the following comments in the previous iterations and your suggestion points were subsequently evaluated and appended to the dataset:
{comment_history}

New experimental data has been obtained giving us currently a dataset of {iteration} experiments:
{trial_data}

**Task**:
 - Reflect on your previous hypotheses and on the entire data.
 - What do we know from the data?
 - Continue live commenting on the optimization progress.
 - What are the subspaces that maximize {target} the most?
 - What are the subspaces that are detrimental and minimize {target}?
 - Improve your previous hypotheses for maximizing {target} in the light of the new comment and learnings. Feel free to also discard some hypotheses that have proven to be performing consistently poorly in previous iterations, and/or propose new ones. Additionally for each hypothesis, give a single point {constraint} you want the Bayesian Op

### jsonline2prompt_template

1. 把 jsonline 放置到 src/prompts 的 test_prompts.json 文件夹下面，注意格式
2. 通过 prompt manager加载打印，其内容应该和上面👆一致

In [6]:
from src.prompts.base import PromptManager

pm = PromptManager()
print(pm.get(key="optimization_loop"))

Error loading agent_prompts.json: Expecting value: line 1 column 1 (char 0)
Error loading llm_prompts.json: Expecting value: line 1 column 1 (char 0)
The Bayesian Optimization is in progress and is currently at iteration {iteration}.
You generated the following comments in the previous iterations and your suggestion points were subsequently evaluated and appended to the dataset:
{comment_history}

New experimental data has been obtained giving us currently a dataset of {iteration} experiments:
{trial_data}

**Task**:
 - Reflect on your previous hypotheses and on the entire data.
 - What do we know from the data?
 - Continue live commenting on the optimization progress.
 - What are the subspaces that maximize {target} the most?
 - What are the subspaces that are detrimental and minimize {target}?
 - Improve your previous hypotheses for maximizing {target} in the light of the new comment and learnings. Feel free to also discard some hypotheses that have proven to be performing consistent

### prompt_template->prompt

1. 自己写一些测试 meta_dict，包含 prompt_template 中的所有{}中的内容，meta_dict 为 python dict
2. 通过 PromptManager 的 format方法测试能否 format 通过

In [10]:
meta_dict = {
    "comment_history": "3242",
    "target": "afasdf",
    "trial_data": "sdfasdf",
    "iteration": 4,
    "constraint": "asdfasdf",
}
print(pm.format("optimization_loop", **meta_dict))

The Bayesian Optimization is in progress and is currently at iteration 4.
You generated the following comments in the previous iterations and your suggestion points were subsequently evaluated and appended to the dataset:
3242

New experimental data has been obtained giving us currently a dataset of 4 experiments:
sdfasdf

**Task**:
 - Reflect on your previous hypotheses and on the entire data.
 - What do we know from the data?
 - Continue live commenting on the optimization progress.
 - What are the subspaces that maximize afasdf the most?
 - What are the subspaces that are detrimental and minimize afasdf?
 - Improve your previous hypotheses for maximizing afasdf in the light of the new comment and learnings. Feel free to also discard some hypotheses that have proven to be performing consistently poorly in previous iterations, and/or propose new ones. Additionally for each hypothesis, give a single point asdfasdf you want the Bayesian Optimizer to try.
 - Important: Only provide your 

### 函数测试！！！
放在具体使用的函数中测试
1. 封装正确的 meta_dict，需要包含上面👆meta_dict 的所有键
2. 使用.format 方法，打印 format 后的 prompt
3. 正确封装好的 format，输入目标函数，查看 llm 输出是否符合预期